In [1]:
import os, sys
curr_dir = os.getcwd()# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
parent_dir = os.path.dirname(curr_dir)
sys.path.insert(0, parent_dir)

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from scripts.model_explainer import pipeline_model_explainability
# Run the pipeliner for model explainability processses
pipeline_model_explainability()